## Fine tuning a Pretrained BERT for spell correction tasks.

### Data download and setup

We will be using a general spell correction dataset available [here](https://github.com/mhagiwara/xfspell/blob/master/data/fce/fce-split.us.tsv). This dataset is an induced spell errors using general english sentences. Download the file and store it in the local notebook instance.

In [8]:
import pandas as pd

In [10]:
ScheckDF = pd.read_csv("xfspell/data/fce/fce-split.us.tsv",sep="\t",header=None,names=["input_text","target_text"])

In [53]:
ScheckDF.head().values

array([['I WANT TO THAK YOU FOR PREPARING SUCH A GOOD PROGRAMME FOR US AND ESPECIALLY FOR TAKING US ON THE RIVER TRIP TO GREENWICH.',
        'I WANT TO THANK YOU FOR PREPARING SUCH A GOOD PROGRAMME FOR US AND ESPECIALLY FOR TAKING US ON THE RIVER TRIP TO GREENWICH.'],
       ['I WOULD LIKE TO KNOW IF THERE IS ANY CHANCE OF CHANGING THE PROGRAMME BECAUSE WE HAVE FOUND A VERY INTERESTING ACTIVITY TO DO ON TUESDAY 14 MARCH.',
        'I WOULD LIKE TO KNOW IF THERE IS ANY CHANCE OF CHANGING THE PROGRAMME BECAUSE WE HAVE FOUND A VERY INTERESTING ACTIVITY TO DO ON TUESDAY 14 MARCH.'],
       ['IT INVOLVES VISITING THE LONDON FASHION AND LEISURE SHOW AT THE CENTRAL EXHIBITION HALL.',
        'IT INVOLVES VISITING THE LONDON FASHION AND LEISURE SHOW AT THE CENTRAL EXHIBITION HALL.'],
       ["I THINK IT'S A GREAT OPPORTUNITY TO MAKE GREATER USE OF OUR KNOWLEDGE OF THE ENGLISH LANGUAGE.",
        "I THINK IT'S A GREAT OPPORTUNITY TO MAKE GREATER USE OF OUR KNOWLEDGE OF THE ENGLISH LANGUAGE."],

In [36]:
trainDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8498 entries, 0 to 8499
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   input_text   8498 non-null   object
 1   target_text  8498 non-null   object
dtypes: object(2)
memory usage: 199.2+ KB


####  Divide the data into train and eval set and remove null values if any.

In [16]:
trainDF = ScheckDF[:8500]
evalDF = ScheckDF[8500:]

In [32]:
trainDF = trainDF.dropna(how='any',axis=0) 
evalDF = evalDF.dropna(how='any',axis=0) 

In [ ]:
print("size of training data {}".format(len(trainDF.index)))
print("size of eval data {}".format(len(evalDF.index)))

### Implementation details

The Idea is to use a Seq2Seq Architecture with both inputs and targets as text sequences. For our dataset we will use the incorrect sentences as inputs and the correct sentences as outputs. We will be further using a encoder-decoder architecture with fine tuning with Bert/Roberta for both encoder and decoder networks.

We will be using a python library called "simpletransformers" which uses the "Transformers" library from hugging faces to build more high level usable API's for certain language tasks. More information can be found [here](https://simpletransformers.ai/)

#### Install the simple transformers library

In [1]:
! pip install simpletransformers

     |████████████████████████████████| 214 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 662 kB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 1.7 MB 32.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 43.5 MB/s eta 0:00:01
     |████████████████████████████████| 7.4 MB 68.6 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 16.4 MB/s eta 0:00:01
     |████████████████████████████████| 308 kB 64.3 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 12.4 MB/s eta 0:00:01
  Using cached GitPython-3.1.9-py3-none-any.whl (159 kB)
     |████████████████████████████████| 1.3 MB 69.4 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 59.9 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 63.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 62.6

  Using cached smmap-3.0.4-py2.py3-none-any.whl (25 kB)


  Created wheel for seqeval: filename=seqeval-1.2.1-py3-none-any.whl size=16167 sha256=4dc407db4f5009fa6e6b695d724c104683d3e37945ab8bfddd9e3744a4147cc2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/41/ac/96/e9d5bceb83600d09ba2ca99693befff9b47bc35334943da786
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6489 sha256=ebeecb9acfb5accbd06935ebf43b9021fb086287bde7f27bb4248cccaaaab9e3
  Stored in directory: /home/ec2-user/.cache/pip/wheels/44/3a/ab/102386d84fe551b6cedb628ed1e74c5f5be76af8b909aeda09
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21495 sha256=5dede3c08ddd83fba9aa751950b8736a6959907ec2d149797b41b105e32a0b76
  Stored in directory: /home/ec2-user/.cache/pip/wheels/59/9a/1d/3f1afbbb5122d0410547bf9eb50955f4a7a98e53a6d8b99bd1
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=77dfdd34a73938970f61b675f45a7357d59b4565be6ed8adbe2800f5442bbf2e
  Stored in directory: /home/

### Train a Model with roberta as encoder and Bert as decoder.

This is a default setting from the library. The following rules currently apply to Encoder-Decoder models:

1. The decoder must be a bert model.
2. The encoder can be one of [bert, roberta, distilbert, camembert, electra].
3. The encoder and the decoder must be of the same "size". (E.g. roberta-base encoder and a bert-base-uncased decoder)

We will be using a batch size of 16 set to 10 epochs and GPU(preferably P3's) to run the below code. 

In [2]:
from simpletransformers.seq2seq import Seq2SeqModel
import logging
import pandas as pd
import sklearn


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [35]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 15,
    "train_batch_size": 16,
    "num_train_epochs": 10,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "max_length": 15,
    "manual_seed": 4,
}

encoder_type = "roberta"

model = Seq2SeqModel(
     encoder_type,
    "roberta-base",
    "bert-base-cased",
    args=model_args,
    use_cuda=True,
)

model.train_model(trainDF)



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.c

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f0673ada0f0>>
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/pytho

INFO:simpletransformers.seq2seq.seq2seq_model: Training started


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-2000
/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-4000


INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/


INFO:simpletransformers.seq2seq.seq2seq_model: Training of roberta-base-bert-base-cased model complete. Saved to outputs/.


(5320, 0.709893412254637)

### Evaluate the model on evalset by taking 1000 sentences

In [38]:
results = model.eval_model(evalDF[:1000])

INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.seq2seq.seq2seq_utils: Saving features into cached file cache_dir/roberta-base-bert-base-cased_cached_151000


INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.488968826815486}


### Run predictions on few unseen sentences

In [62]:
print(model.predict(["Wht is your name"]))

['What is your name?....']
